# Contraction Hierarchy Query SDK - Testing Notebook

This notebook demonstrates and tests the **new HTTP-based SDK** which connects to the persistent C++ routing server.

**Prerequisite**: Ensure the `routing-server` is running on port 8080.

In [1]:
import sys
import time
sys.path.insert(0, '/home/kaveh/projects/routing-pipeline')

from api.ch_query import CHQueryEngineFactory

print("✓ Imports successful!")

✓ Imports successful!


## 1. Initialize Query Engine

Connect to the local routing server (default: http://localhost:8080).

In [2]:
# Initialize Factory
factory = CHQueryEngineFactory(server_url="http://localhost:8080")

# Register dataset (Configuration is now handled on the server, but we register the name locally context)
dataset_name = "Burnaby"
factory.register_dataset(dataset_name)

# Get Engine
engine = factory.get_engine(dataset_name)
print(f"✓ Connected to engine for dataset: {engine.dataset_name}")

✓ Connected to engine for dataset: burnaby


## 2. Test Route Computation

Compute a route using Lat/Lon coordinates (Burnaby example).

In [3]:
# Coordinates for a trip in Burnaby
start_lat, start_lon = 49.2827, -122.9781
end_lat, end_lon = 49.2500, -122.9500

print(f"Requesting route: ({start_lat}, {start_lon}) -> ({end_lat}, {end_lon})")

start_time = time.time()
result = engine.compute_route_latlon(
    start_lat=start_lat,
    start_lng=start_lon,
    end_lat=end_lat,
    end_lng=end_lon
)
elapsed_ms = (time.time() - start_time) * 1000

if result.success:
    print(f"\n✓ Success!")
    print(f"  Distance: {result.distance:.2f} meters")
    print(f"  Path Nodes: {len(result.path)} edges")
    print(f"  GeoJSON Points: {len(result.geojson['geometry']['coordinates']) if result.geojson else 0}")
    print(f"  Total Latency (Client+Net+Server): {elapsed_ms:.2f} ms")
else:
    print(f"✗ Failed: {result.error}")

Requesting route: (49.2827, -122.9781) -> (49.25, -122.95)

✓ Success!
  Distance: 157.10 meters
  Path Nodes: 50 edges
  GeoJSON Points: 180
  Total Latency (Client+Net+Server): 4.16 ms


## 3. Performance Test

Run a quick benchmark to verify the cached client performance.

In [4]:
iterations = 50
latencies = []

print(f"Running {iterations} queries...")
for _ in range(iterations):
    t0 = time.time()
    result = engine.compute_route_latlon(
        start_lat=start_lat,
        start_lng=start_lon,
        end_lat=end_lat,
        end_lng=end_lon
    )
    latencies.append((time.time() - t0) * 1000)

avg_lat = sum(latencies) / len(latencies)
min_lat = min(latencies)
max_lat = max(latencies)

print(f"\nResults:")
print(f"  Average: {avg_lat:.2f} ms")
print(f"  Min: {min_lat:.2f} ms")
print(f"  Max: {max_lat:.2f} ms")
print(f"  Throughput: {1000/avg_lat:.1f} RPS")

Running 50 queries...

Results:
  Average: 2.43 ms
  Min: 1.82 ms
  Max: 4.02 ms
  Throughput: 411.5 RPS
